In [ ]:
from genparse.fst import FST, EPSILON
from genparse import Float
from genparse.util import LarkStuff, bpe_wfst
from arsenal import iterview
from collections import Counter

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')
print(f'token vocabulary size: {tokenizer.vocab_size}')
T = [(token_id, tokenizer.decode([token_id])) for token_id in range(tokenizer.vocab_size)]

In [ ]:
import random
random.seed(8675309)
#S = random.sample(T, 1000)
S = T

In [ ]:
b2c = bpe_wfst(S).renumber
len(b2c.states)

In [ ]:
b = tokenizer.encode('SELECT * FROM data')
b

In [ ]:
FST.from_string(tuple(b), Float) @ b2c

In [ ]:
lark_stuff = LarkStuff(r"""

start: [ KEYWORD | INTEGER | STAR | NAME | BOOLEAN | WS ] EOS

EOS: "</s>"
NAME: /[A-Za-z]+/
KEYWORD: "SELECT" | "WHERE" | "FROM"
INTEGER: /[1-9][0-9]*/
BOOLEAN: "true"  | "false"
STAR: "*"
WS: /\s+/

""")

c2t = lark_stuff.transducer(ignore='')

In [ ]:
x = 'Thicken purple secret warehouse'
c = FST.from_string(x, Float).renumber

In [ ]:
#c

In [ ]:
tokenizer.encode(x)

In [ ]:
FST.from_pairs([
    ('aaaaa', 'bb'),
    ('a', 'ccc'),
    ('', 'empty'),
    ('nonempty', ''),
], Float)

In [ ]:
def bpe2term_approx(bpe):
    
    # approximate the transducer using a single canonical path;
    # UPDATE: the unpruned answer should match this - it's the uncertainty over bpe that's tricky
    c = tuple(([b], tokenizer.decode([b])) for b in bpe)
    print(c)
    tmp = FST.from_pairs([([], '')], Float)
    for pair in c:
        tmp = tmp * FST.from_pairs([pair], Float)
        
    return tmp
    # TODO: approximate this transducer by a canonical path
#    return c2t(c, None).trim.epsremove.trim

In [ ]:
t = bpe2term_approx(tokenizer.encode('SELECT * FROM data'))
t

In [ ]:
t.epsremove.trim

In [ ]:
FST.from_string(tuple(b), Float) @ b2c

In [ ]:
#t.to_cfg().cnf.language(50)

In [ ]:
e = b2c @ c

In [ ]:
def about(m):
    print(f"states: {len(e.states)}, trim: {len(e.trim.states)} (access: {len(e.accessible())} co-access: {len(e.co_accessible())})")

In [ ]:
about(e)

In [ ]:
e.trim

In [ ]:
#e.trim.renumber.to_cfg().trim().cnf.language(55)

In [ ]:
e.trim.renumber.project(0).epsremove.trim

In [ ]:
e.trim.renumber.project(0).epsremove.trim.to_cfg().trim().cnf.language(6)

In [ ]:
#tmp = c2t('SELECT', None).trim.epsremove.trim
#tmp

In [ ]:
#tmp = c2t('SELECT\n\t\r FROM 1 2', None).trim.epsremove.trim#.min
#tmp

In [ ]:
#tmp = c2t('SELECT\nFROM', ('KEYWORD', 'KEYWORD'))
#tmp

In [ ]:
# random.seed(8675309)
# bpe_small = random.sample(T, 10)
# b2c_small = bpe_wfst(bpe_small).renumber
# len(b2c_small.states)

In [ ]:
#bpe_small

In [ ]:
#b2c_small

In [ ]:
#c = FST.from_string('zon ankles Turtles purple', Float).renumber

In [ ]:
#len((b2c_small @ c).states), len((b2c_small @ c).trim.states)

In [ ]:
#(b2c_small @ c).trim

In [ ]:
# N = lambda x: abs(hash(x)) % 3
# A = lambda x: abs(hash(x)) % 2
# B = lambda x: abs(hash(x)) % 7

# bbb = b2c_small.coarsen(N, A, B)
# ccc = c.coarsen(N = lambda x: abs(hash(x)) % 5, A = B, B = B)

In [ ]:
#bbb

In [ ]:
#ccc

In [ ]:
#monster = b2c @ c2t

In [ ]:
#len(monster.states), len(monster.trim.states)

In [ ]:
#monster_baby = FST.from_string((5188,), Float).renumber @ monster 

In [ ]:
#len(monster_baby.states), len(monster_baby.trim.states)

In [ ]:
x = 'SELECT * FROM data'

In [ ]:
c = FST.from_string(x, Float).renumber
c

In [ ]:
query = b2c @ c

In [ ]:
query.trim

In [ ]:
len(query.states), len(query.trim.states)

In [ ]:
terminals = (c @ lark_stuff.transducer()).trim
terminals

In [ ]:
list(lark_stuff.lex(x))

In [ ]:
bpe2terminal = query.trim @ terminals

In [ ]:
len(bpe2terminal.states), len(bpe2terminal.trim.states)

In [ ]:
bpe2terminal.trim